In [60]:
import time
import os
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# descomentar estas 4 próximas linhas para usar o navegador invisível
#gecko = '/home/andre/Downloads/geckodriver'
#driver = webdriver.FirefoxOptions()
#driver.add_argument('-headless')
#driver = webdriver.Firefox(executable_path=gecko, options=driver)

def abre_jogos():

    html = driver.execute_script("return document.documentElement.outerHTML")
    page = bs(html, 'html.parser')

    # contando o número de jogos ao vivo
    tam = int(driver.find_element_by_css_selector('.js-sport-games-live-count > span:nth-child(2)').text[1])
    print(f'número de jogos ao vivo {tam}')
    excecao = False
    
    # Percorrendo todos os jogos da rodada
    num_aba = 0
    for j in range(tam):
        try:

            # clicando nos jogos e abrindo em nova aba
            
            html = driver.execute_script("return document.documentElement.outerHTML")
            page = bs(html, 'html.parser')

            pagina = page.find_all('',{'class':'js-event-list-tournament-events'})[j].find('a').get('href')
            pagina = 'https://www.sofascore.com' + pagina 

            # abre a nova aba
            driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't')

            # muda de aba   
            driver.switch_to.window(driver.window_handles[num_aba + 1])

            # acessa nova página
            driver.get(pagina)
            time.sleep(7)
            
             # verificando se o jogo possui estatísticas
            try:                                             
                teste = driver.find_element_by_css_selector('.statistics-container > h3:nth-child(1)').text
                excecao = False
            except:
                
                # fecha a aba
                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'w')
                excecao = True
                
                # mudar para a aba aberta antes - página principal 
                driver.switch_to.window(driver.window_handles[0])

                # pula para o próximo jogo
                continue
            
            # mudar para a aba aberta antes - página principal 
            driver.switch_to.window(driver.window_handles[0])
            
        except:
            
            print('exceção')
            # mudar para a aba aberta antes - página principal 
            driver.switch_to.window(driver.window_handles[0])
            continue
        
        num_aba += 1
        
        # verificando se precisou fechar alguma pagina e atualizar o nº da aba
        if excecao == True:
            num_aba -= 1
    return num_aba
            

def scanner(num_jogos):
    
    possebola = 'Posse de bola'
    chutes = 'Finalizações'
    chutesnogol = 'Finalizações no gol'
    chutesforagol = 'Finalizações para fora'
    chutesbloqueados = 'Chutes travados'
    corner = 'Escanteios'
    impedimentos = 'Impedimentos'
    faltas = 'Faltas'
    cartoesa = 'Cartões amarelos'
    cartoesv = 'Cartões vermelhos'
    grandeschances = 'Grandes chances'
    grandeschancesperdidas = 'Grandes chances perdidas'
    chutesnatrave = 'Finalizações na trave'
    contraataques = 'Contra-ataques'
    chutesdentroarea = 'Finalizações de dentro da área'
    chutesforaarea = 'Finalizações de fora da área'
    defesasgoleiro = 'Defesas do goleiro'
    passes = 'Passes'
    passescertos = 'Passes certos'
    bolaslongas = 'Bolas longas'
    cruza = 'Cruzamentos'
    dribles = 'Dribles'
    posseperdida = 'Possession lost'
    duelosganhos = 'Duelos ganhos'
    disputasaereasvencidas = 'Disputas aéreas vencidas'
    desarmes = 'Desarmes'
    intercept = 'Interceptações'
    cortes = 'Cortes'
    
    mingol = []
    resultadopormin = []
    resultadomin = []
    goltime = []
    golxtime1min = []
    golxtime2min = []
    
    for i in range(num_jogos):
        
        # mudar para a aba aberta antes - página principal 
        driver.switch_to.window(driver.window_handles[i + 1])
    
        # Coletando informações do jogo                            

        pais = driver.find_element_by_css_selector('.js-breadcrumb-category-link').text
        campeonato = driver.find_element_by_css_selector('.js-breadcrumb-tournament-link').text
        jogo = driver.find_element_by_css_selector('.js-event-page-event-name').text
        
        minuto_atual = driver.find_element_by_css_selector('.js-event-status-description').text
        resultado_parcial = driver.find_element_by_css_selector('.h1').text
        resultado_parcial = resultado_parcial[0] + 'x' + resultado_parcial[4]                 

        print('\n')
        print(pais)                             
        print(campeonato)
        print(jogo)
        print(f'Resultado Parcial {resultado_parcial}')
        print(f'Minuto {minuto_atual}')

        # rolar a página mais para baixo
        body = driver.find_element_by_css_selector('body')
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1) 

        html = driver.execute_script("return document.documentElement.outerHTML")
        page = bs(html, 'html.parser')


        # aqui começa a coleta de minuto de 1o gol 2o gol, placar em 5 e 5 min, quem fez o 1o
        # o 2o gol do jogo              

        # contando quantos gols aconteceram no jogo
        tam_placar = len(page.find_all('',{'class':'incident__goal'}))

        # contando quantos gols cada time fez
        tam_minutogol = len(page.find('div',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '}))
        tam_minutogol2 = len(page.find('div',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '}))

        # coletando o minuto dos gols

        # minuto que o placar mudou e novo placar
        for totalgols in range(tam_placar):
            placarpormin = page.find_all('',{'class':'incident__goal'})[totalgols].text.strip().replace(' - ','x')
            resultadopormin.append(placarpormin)

            # pegando na posição 0 o placar e na próxima o minuto e assim por diante
            # o minuto ele pega no trecho abaixo
            resultadomin.append(placarpormin)


        # gols do time 1
        for minutosgols in range(tam_minutogol):
            try:
                teste = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '})[minutosgols].find('span',{'class':'incident__goal'}).text.strip().replace(' - ','x')
                minutogol = int(page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '})[minutosgols].find('',{'cell__content incident__time'}).text.strip().split("'")[0])
                mingol.append(minutogol)                                                                                                                         

                # verificando os gols depois dos 90 min
                if mingol[len(mingol) - 1] == 90:
                    try:                                                                                                                                                                                                      
                        mingolacrescimo = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '})[minutosgols].find('',{'cell__content incident__time'}).find('sup').text.strip().split("+")[1] 
                        mingolacrescimo = int(mingolacrescimo) + int(mingol[len(mingol) - 1])
                        mingol[len(mingol) - 1] = mingolacrescimo
                    except:
                        pass

                resultadomin.append(mingol[len(mingol) - 1])

                # goltime armazena o placar e quem fez o gol
                goltime.append(teste)
                goltime.append('1')

                # verifica o minuto do 1o, 2o, 3o gol do time 1
                golxtime1min.append(mingol[len(mingol) - 1])

            except:
                continue

        # gols do time 2       
        for minutosgols2 in range(tam_minutogol2):
            try:
                teste2 = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '})[minutosgols2].find('span',{'class':'incident__goal'}).text.strip().replace(' - ','x')
                minutogol = int(page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '})[minutosgols2].find('',{'cell__content incident__time'}).text.strip().split("'")[0])
                mingol.append(minutogol)

                # verificando os gols depois dos 90 min
                if mingol[len(mingol) - 1] == 90:
                    try:                                                                                                                                                                                                      
                        mingolacrescimo = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '})[minutosgols2].find('',{'cell__content incident__time'}).find('sup').text.strip().split("+")[1] 
                        mingolacrescimo = int(mingolacrescimo) + int(mingol[len(mingol) - 1])
                        mingol[len(mingol) - 1] = mingolacrescimo
                    except:
                        pass


                resultadomin.append(mingol[len(mingol) - 1])

                # goltime armazena o placar e quem fez o gol
                goltime.append(teste2)
                goltime.append('2')

                # verifica o minuto do 1o, 2o, 3o gol do time 2
                golxtime2min.append(mingol[len(mingol) - 1])

            except:
                continue   

        # vamos precisar saber o tamanho desse array
        tam_goltime = len(goltime)

        # variáveis auxiliares que receberam o placar por min de gol e o min dos gols sendo ordenadas
        resultadopormin.reverse()
        golxtime1min.reverse()
        golxtime2min.reverse()
        mingol.sort()
        
        # testando se o jogo foi 0x0
        if resultado_parcial == '0x0':
            mingol.append('-1')
            resultadopormin.append('0x0')
            goltime.append('null')
            resultadomin.append('0x0')
            resultadomin.append('-1')

        # Pegando as estatísticas ht e ft do jogo
        for blocos in range(7):
            bloco1 = str(1 + blocos)
            for estatisticas in range(7):
                bloco2 = str(1 + estatisticas)                  
                try:                                                 
                    nome_dado = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(2) > div:nth-child(1)').text
                    if nome_dado.strip() == possebola:

                        possebolacasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        possebolavisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutes:

                        chutescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutesnogol:

                        chutesnogolcasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesnogolvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutesforagol:

                        chutesforagolcasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesforagolvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutesbloqueados:

                        chutesbloqueadoscasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesbloqueadosvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == corner: 

                        cornercasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        cornervisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == impedimentos:

                        impedimentoscasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        impedimentosvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == faltas:

                        faltascasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        faltasvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == cartoesa:

                        cartoesacasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        cartoesavisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == cartoesv:

                        cartoesvcasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        cartoesvvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == grandeschances:

                        grandeschancescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        grandeschancesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == grandeschancesperdidas:

                        grandeschancesperdidascasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        grandeschancesperdidasvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutesnatrave:

                        chutesnatravecasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesnatravevisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == contraataques:

                        contraataquescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        contraataquesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutesdentroarea:

                        chutesdentroareacasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesdentroareavisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == chutesforaarea:

                        chutesforaareacasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        chutesforaareavisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == defesasgoleiro:

                        defesasgoleirocasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        defesasgoleirovisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == passes:

                        passescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        passesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == passescertos:

                        passescertoscasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        passescertosvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == bolaslongas:

                        bolaslongascasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        bolaslongasvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == cruza:

                        cruzacasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        cruzavisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == dribles:

                        driblescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        driblesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == posseperdida:

                        posseperdidacasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        posseperdidavisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == duelosganhos:

                        duelosganhoscasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        duelosganhosvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == disputasaereasvencidas:

                        disputasaereasvencidascasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        disputasaereasvencidasvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == desarmes:

                        desarmescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        desarmesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == intercept:

                        interceptcasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        interceptvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                    elif nome_dado.strip() == cortes:

                        cortescasa = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(1) > div:nth-child(1)').text
                        cortesvisitante = driver.find_element_by_css_selector('#statistics-period-ALL > div:nth-child('+ bloco1 +') > div:nth-child('+ bloco2 +') > div:nth-child(3) > div:nth-child(1)').text

                except:
                    continue
                    
        # pegando as estatísticas simples de jogo 
        
        # quantidade de blocos de dados
        tam_stats = len(page.find('',{'class':'statistics-container'}).find_all('',{'class':'stat-group-event'}))
        for tam_stats1 in range(tam_stats):
            try:      
                # quantidade de dados em cada bloco
                tam_blocos = len(page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'}))
                
                # scaneando os dados de cada bloco
                for tam_blocos1 in range(tam_blocos):
                    try:
                        
                        nome_dado = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content'}).text
                        if nome_dado.strip() == possebola:

                            possebolacasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            possebolavisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutes:

                            chutescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutesnogol:

                            chutesnogolcasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesnogolvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutesforagol:

                            chutesforagolcasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesforagolvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutesbloqueados:

                            chutesbloqueadoscasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesbloqueadosvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == corner: 

                            cornercasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            cornervisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == impedimentos:

                            impedimentoscasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            impedimentosvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == faltas:

                            faltascasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            faltasvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == cartoesa:

                            cartoesacasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            cartoesavisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == cartoesv:

                            cartoesvcasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            cartoesvvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == grandeschances:

                            grandeschancescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            grandeschancesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == grandeschancesperdidas:

                            grandeschancesperdidascasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            grandeschancesperdidasvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutesnatrave:

                            chutesnatravecasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesnatravevisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == contraataques:

                            contraataquescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            contraataquesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutesdentroarea:

                            chutesdentroareacasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesdentroareavisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == chutesforaarea:

                            chutesforaareacasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            chutesforaareavisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == defesasgoleiro:

                            defesasgoleirocasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            defesasgoleirovisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == passes:

                            passescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            passesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == passescertos:

                            passescertoscasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            passescertosvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == bolaslongas:

                            bolaslongascasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            bolaslongasvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == cruza:

                            cruzacasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            cruzavisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == dribles:

                            driblescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            driblesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == posseperdida:

                            posseperdidacasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            posseperdidavisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == duelosganhos:

                            duelosganhoscasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            duelosganhosvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == disputasaereasvencidas:

                            disputasaereasvencidascasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            disputasaereasvencidasvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == desarmes:

                            desarmescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            desarmesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == intercept:

                            interceptcasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            interceptvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text

                        elif nome_dado.strip() == cortes:

                            cortescasa = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-home'}).text
                            cortesvisitante = page.find('',{'class':'statistics-container'}).find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__content stat-away'}).text
                    
                    except:
                        continue
                        
            except:
                continue

            

# início do script principal

os.environ['MOZ_FORCE_DISABLE_E10S'] = '1' # lembrar de colocar 1 depois do igual para este uso
url_inicio = 'https://www.sofascore.com'

 # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
driver = webdriver.Firefox() 
driver.get(url_inicio)
time.sleep(1)

driver.find_element_by_css_selector('.js-sport-games-live-count').click()
time.sleep(7)

try:
    num_jogos = abre_jogos() # função para abrir os jogos da rodada
    
    while True:
        scanner(num_jogos) # função para scanear os dados in live dos jogos
except:
    print('Nenhum jogo ao vivo no momento.')
    driver.quit()


Nenhum jogo ao vivo no momento.
